## cross validation

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import pandas as pd
import tensorflow as tf
import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

Using TensorFlow backend.


### data preprocessing

In [2]:
df = pd.read_csv('Artificial_Neural_Networks/Churn_Modelling.csv')
print df.shape
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print X.shape

# remove one dummy variable to avoid dummy variable trap
X = X[:, 1:]
print X.shape

(10000, 14)
(10000, 12)
(10000, 11)


### train test split & feature scaling 

In [3]:
S = lambda *x: [i.shape for i in x]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# print S(X_train,X_test, y_train, y_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print S(X_train,X_test, y_train, y_test)

[(8000, 11), (2000, 11), (8000,), (2000,)]


### build k-fold func ANN
## Dropout and GridSearch (parameter tuning)

In [4]:
from keras.layers import Dropout

from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
# k-fold cross validation
from sklearn.model_selection import GridSearchCV

In [5]:
def build_classifier(opt, P):
    classifier = Sequential()
    # first hidden layer
    # classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units = 6, input_dim=11,  kernel_initializer='uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(p= P))
    
    # second hidden layer
    classifier.add(Dense(units = 6,  kernel_initializer='he_uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(p= P))    
    
    # ouput layer
    classifier.add(Dense(units = 1,  kernel_initializer='he_uniform', activation='sigmoid'))
    # compiling the ANN
    classifier.compile(optimizer= opt, loss='binary_crossentropy', metrics= ['accuracy'])
#     print classifier.summary()
    return classifier

classifier = KerasClassifier(build_fn= build_classifier)

In [6]:
parameters = {
    'batch_size':[32, 28, 36],
    'nb_epoch':[250, 300, 350],
    'opt':['adam','rmsprop'],
    'P':[0.03, 0.02, 0.01]
}

In [ ]:
grid_sesarch = GridSearchCV(estimator= classifier, param_grid= parameters, 
                           scoring= 'accuracy', cv= 10, n_jobs=-1)

In [ ]:
if __name__ == "__main__":
    import sys
    from time import time
    start = time()
    grid = grid_sesarch.fit(X_train, y_train)
    eclipse = time()-start
    print 'used: {:.2f} s'.format(eclipse)
    
    best_parameters = grid.best_params_
    best_accuracy = grid.best_score_

/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.03)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.03)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.03)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.03)`
  import sys
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.03)`
  if sys.path[0] == '':
/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
7200/7200 [==============================] - 0s - loss: 0.5502 - acc: 0.7913     Epoch 2/10

  32/7200 [..............................] - ETA: 0s - loss: 0.5357 - acc: 0.6875
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.5535 - acc: 0.7910     
Epoch 2/10
6592/7200 [==========================>...] - ETA: 0s - loss: 0.4386 - acc: 0.7934
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.4368 - acc: 0.7956     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.4331 - acc: 0.7975     
Epoch 3/10
4384/7200 [=================>............] - ETA: 0s - loss: 0.4217 - acc: 0.7958
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4213 - acc: 0.7956     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4180 - acc: 0.7971     Epoch 4/10

Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4001 - acc: 0.8151     
Epoch 5/10
4800/7200 [====

7200/7200 [==============================] - 0s - loss: 0.3669 - acc: 0.8482     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3697 - acc: 0.8442     
Epoch 9/10
  32/7200 [..............................] - ETA: 0s - loss: 0.3198 - acc: 0.8125
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3667 - acc: 0.8489     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3607 - acc: 0.8492     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3639 - acc: 0.8501     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3687 - acc: 0.8483     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.6269 - acc: 0.7635     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.6288 - acc: 0.7650     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.6303 - acc: 0.7650     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.4789 - a

4416/7200 [=================>............] - ETA: 0s - loss: 0.4254 - acc: 0.7996
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4251 - acc: 0.8046     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.4330 - acc: 0.7962     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3989 - acc: 0.8322     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4097 - acc: 0.8225     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4120 - acc: 0.8224     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4214 - acc: 0.8008     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.4052 - acc: 0.8247     Epoch 6/10

Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.4126 - acc: 0.8083     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3929 - acc: 0.8375     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.3

2432/7200 [=========>....................] - ETA: 0s - loss: 0.3868 - acc: 0.8429
Epoch 10/10
3744/7200 [==============>...............] - ETA: 0s - loss: 0.3827 - acc: 0.8464
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3784 - acc: 0.8497     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3746 - acc: 0.8499     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3741 - acc: 0.8522     
Epoch 10/10
  32/7200 [..............................] - ETA: 71s - loss: 0.6938 - acc: 0.5938
Epoch 2/10
2848/7200 [==========>...................] - ETA: 0s - loss: 0.6321 - acc: 0.7753
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.4248 - acc: 0.7960     
Epoch 3/10
3328/7200 [============>.................] - ETA: 0s - loss: 0.6235 - acc: 0.7746
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.4002 - acc: 0.8128     
Epoch 2/10
Epoch 4/10
7200/7200 [==============================] - 0s - loss:

...

7200/7200 [==============================] - 3s - loss: 0.5255 - acc: 0.7924     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4280 - acc: 0.8117     
Epoch 3/10
1056/7200 [===>..........................] - ETA: 1s - loss: 0.4051 - acc: 0.8248

/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.02)`
  import sys


1440/7200 [=====>........................] - ETA: 1s - loss: 0.4215 - acc: 0.8174

/home/saya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.02)`
  if sys.path[0] == '':


  32/7200 [..............................] - ETA: 425s - loss: 0.6800 - acc: 0.6875
Epoch 4/10
4096/7200 [================>.............] - ETA: 2s - loss: 0.5799 - acc: 0.7891
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4057 - acc: 0.8310     
Epoch 5/10
7200/7200 [==============================] - 3s - loss: 0.5273 - acc: 0.7906     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4265 - acc: 0.8132     
Epoch 3/10
1664/7200 [=====>........................] - ETA: 1s - loss: 0.4252 - acc: 0.8119
Epoch 3/10
4000/7200 [===============>..............] - ETA: 2s - loss: 0.5813 - acc: 0.7897
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.4184 - acc: 0.8275     
Epoch 4/10
3200/7200 [============>.................] - ETA: 1s - loss: 0.4154 - acc: 0.8203
Epoch 2/10
3904/7200 [===============>..............] - ETA: 0s - loss: 0.4282 - acc: 0.8099
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.4112 - acc

In [13]:
best_parameters

{'P': 0.01, 'batch_size': 28, 'nb_epoch': 300, 'opt': 'adam'}

In [ ]:
best_accuracy

In [ ]:
# GridSearchCV?

In [ ]:
eclipse

In [ ]:
from time import ctime
with open('result_4.txt', 'w+') as fl:
    fl.write(str(best_parameters)+'\n')
    fl.write(str(best_accuracy)+'\n')
    fl.write(str(eclipse)+'\n')
    fl.write(str(ctime())+'\n')

In [ ]:
# if __name__ == "__main__":
#     from time import time
#     start=time()
#     accuracies = cross_val_score(estimator = classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
#     eclipse = time()-start
#     print 'used: {:.2f} s'.format(eclipse)

In [ ]:
# accuracies

In [ ]:
# accuracies.mean()

In [ ]:
# accuracies.std()

### Predicting a single new observation